# Prep Upper Colorado use-case.

This a subset of the NHM prepped from a Bandit pull by Parker Norton.  Using this as a
test case for forecasting.

In [1]:
import geopandas as gpd
import requests
import pandas as pd
import rioxarray
from pathlib import Path
import numpy as np

In [3]:
root_dir = '/mnt/d/oNHM/NHM_PRMS_TEST/'

In [4]:
root_dir

'/mnt/d/oNHM/NHM_PRMS_TEST/'

In [5]:
def read_param_values(dir, field):
    filename = Path(dir) / "myparam.param"
    values = []  # List to store the elevation values
    start_collecting = False  # Flag to start collecting values
    count_values = 0  # To count the values read
    
    with open(filename, 'r') as file:
        for line in file:
            stripped_line = line.strip()  # Remove any leading/trailing whitespace
            
            if stripped_line == field:
                # Skip the next four lines after 'hru_elev'
                for _ in range(4):
                    next(file)
                start_collecting = True  # Set flag to start collecting after skipping
                continue
            
            if stripped_line == '####' and start_collecting:
                break  # Stop reading if we hit #### after starting to collect
            
            if start_collecting:
                try:
                    # Convert the line to a float and add to the list
                    value = float(stripped_line)
                    values.append(value)
                    count_values += 1
                except ValueError:
                    continue  # If conversion fails, skip the line
        print(f"Read {count_values} {field} values")
    return np.array(values)

In [6]:
gdf = gpd.read_file(root_dir + "GIS/model_nhru.shp")
gdf

,nhm_id,model_hru_,geometry
0,104250,1,"POLYGON ((-1877954.943 1354110.102, -1877970.6..."
1,104271,2,"POLYGON ((-1877986.138 1354095.121, -1877985.2..."
2,104288,3,"MULTIPOLYGON (((-1877954.943 1354110.102, -187..."
3,104320,4,"POLYGON ((-1906094.918 1380104.815, -1906065.1..."
4,104321,5,"MULTIPOLYGON (((-1916474.820 1374164.856, -191..."
5,105705,6,"MULTIPOLYGON (((-1904955.106 1376864.847, -190..."
6,105706,7,"POLYGON ((-1901925.143 1380494.889, -1901924.9..."
7,105709,8,"POLYGON ((-1884644.690 1393635.013, -1884645.2..."
8,105711,9,"POLYGON ((-1880235.020 1375005.155, -1880294.7..."


In [7]:
# Find invalid geometries
invalid_geometries = gdf[~gdf.is_valid]
print(f"Found {len(invalid_geometries)} invalid geometries.")

# Correct invalid geometries by explicitly targeting the original DataFrame
gdf.loc[~gdf.is_valid, 'geometry'] = gdf.loc[~gdf.is_valid, 'geometry'].apply(lambda geom: geom.buffer(0))

# Re-check for invalid geometries after the correction
invalid_geometries_after = gdf[~gdf.is_valid]
print(f"Found {len(invalid_geometries_after)} invalid geometries after correction.")

Found 1 invalid geometries.
Found 0 invalid geometries after correction.


In [8]:
gdf.crs

<Projected CRS: ESRI:102039>
Name: USA_Contiguous_Albers_Equal_Area_Conic_USGS_version
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [9]:
gdf = gdf.to_crs("EPSG:4326")
gdf

,nhm_id,model_hru_,geometry
0,104250,1,"POLYGON ((-116.48952 33.42621, -116.48969 33.4..."
1,104271,2,"POLYGON ((-116.48982 33.42602, -116.48981 33.4..."
2,104288,3,"MULTIPOLYGON (((-116.48952 33.42621, -116.4898..."
3,104320,4,"POLYGON ((-116.84811 33.59911, -116.84759 33.5..."
4,104321,5,"MULTIPOLYGON (((-116.83083 33.57339, -116.8308..."
5,105705,6,"MULTIPOLYGON (((-116.82840 33.57302, -116.8305..."
6,105706,7,"POLYGON ((-116.80487 33.61063, -116.80466 33.6..."
7,105709,8,"POLYGON ((-116.65250 33.75895, -116.65223 33.7..."
8,105711,9,"POLYGON ((-116.56219 33.60458, -116.56241 33.6..."


## Step 2: Sort the DataFrame by 'nhm_id'
Looks like data are sorted already but we'll sort anyway. CBH data supplied to this model should be sorted by nhm_id.

In [10]:
gdf_sorted = gdf.sort_values(by="nhm_id")

## Step 3: Export "nhm_id" to a file as read from the .param file.

In [11]:
nhm_id = read_param_values(root_dir, "nhm_id")
df = pd.DataFrame(nhm_id.astype(int))
df.to_csv(f'{root_dir}/nhm_id', index=False, header=False)

Read 9 nhm_id values


## Step 4: Export "hru_lat.csv" "hru_lon.csv" to a file as read from the .param file.
These are used when converting hru results as .csv file to netcdf output.  The lat/lon give some spatial context to the
netcdf output files.

In [12]:
hru_lat = read_param_values(root_dir, "hru_lat")
df = pd.DataFrame(hru_lat)
df.to_csv(f'{root_dir}/hru_lat.csv', index=False, header=False)

hru_lon = read_param_values(root_dir, "hru_lon")
df = pd.DataFrame(hru_lon)
df.to_csv(f'{root_dir}/hru_lon.csv', index=False, header=False)

Read 9 hru_lat values
Read 9 hru_lon values


## 5 Get elevations - used for calculating humidity from cfsv2 data.

In [13]:

elevations = read_param_values(root_dir, "hru_elev")
df = pd.DataFrame(elevations)
df.to_csv(f'{root_dir}elevation.csv', index=False, header=False)

Read 9 hru_elev values


## 6 Get segment lat/lon.  
The param db does not contain hru_lon generally and not hru_lat if there is no stream temp solved.  So rather than
using the db to extract those values we need to get them from the geometry file.

In [14]:
gdf = gpd.read_file(root_dir + "GIS/model_nsegment.shp")
gdf

,nhm_seg,model_seg_,geometry
0,56129,3,"LINESTRING (-1880479.934 1374411.164, -1880578..."
1,56140,2,"LINESTRING (-1902007.018 1380563.223, -1902064..."
2,56162,1,"LINESTRING (-1905174.643 1376946.810, -1905191..."
3,56285,4,"LINESTRING (-1884257.015 1357429.218, -1884141..."


In [15]:
invalid_geometries = gdf[~gdf.is_valid]
print(f"Found {len(invalid_geometries)} invalid geometries.")

Found 0 invalid geometries.


In [16]:
gdf_sorted = gdf.sort_values(by="nhm_seg")
gdf_sorted['nhm_seg'].to_csv(
    f'{root_dir}nhm_seg', index=False, header=False
)
centroids = gdf_sorted.to_crs(5070).geometry.centroid
centroids_ll = centroids.to_crs(4326)
lats = centroids_ll.y
lons = centroids_ll.x
lats.to_csv(f'{root_dir}seg_lat.csv', index=False, header=False)
lons.to_csv(f'{root_dir}seg_lon.csv', index=False, header=False)

## Part 7 Generate gridmet weights.

In [17]:
from gdptools import WeightGen
from gdptools import AggGen
from gdptools import ClimRCatData


In [18]:
gdf = gpd.read_file(root_dir + "GIS/model_nhru.shp")
gdf

,nhm_id,model_hru_,geometry
0,104250,1,"POLYGON ((-1877954.943 1354110.102, -1877970.6..."
1,104271,2,"POLYGON ((-1877986.138 1354095.121, -1877985.2..."
2,104288,3,"MULTIPOLYGON (((-1877954.943 1354110.102, -187..."
3,104320,4,"POLYGON ((-1906094.918 1380104.815, -1906065.1..."
4,104321,5,"MULTIPOLYGON (((-1916474.820 1374164.856, -191..."
5,105705,6,"MULTIPOLYGON (((-1904955.106 1376864.847, -190..."
6,105706,7,"POLYGON ((-1901925.143 1380494.889, -1901924.9..."
7,105709,8,"POLYGON ((-1884644.690 1393635.013, -1884645.2..."
8,105711,9,"POLYGON ((-1880235.020 1375005.155, -1880294.7..."


In [19]:
climater_cat = "https://github.com/mikejohnson51/climateR-catalogs/releases/download/March-2024/catalog.parquet"
cat = pd.read_parquet(climater_cat)
cat.head()

,id,asset,URL,type,varname,variable,description,units,model,ensemble,...,Y1,Yn,resX,resY,ncols,nrows,crs,toptobottom,tiled,dim_order
0,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_ACCESS1-0_rcp45_r1i1p1,pr,Precipitation,mm/d,ACCESS1-0,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
1,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_ACCESS1-0_rcp85_r1i1p1,pr,Precipitation,mm/d,ACCESS1-0,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
2,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_bcc-csm1-1_rcp26_r1i1p1,pr,Precipitation,mm/d,bcc-csm1-1,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
3,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_bcc-csm1-1_rcp45_r1i1p1,pr,Precipitation,mm/d,bcc-csm1-1,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
4,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_bcc-csm1-1_rcp60_r1i1p1,pr,Precipitation,mm/d,bcc-csm1-1,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX


In [20]:
_id = "gridmet"
_varname = "tmmx"

# an example query returns a pandas dataframe.
tc = cat.query(
    "id == @_id & variable == @_varname"
)
tc

,id,asset,URL,type,varname,variable,description,units,model,ensemble,...,Y1,Yn,resX,resY,ncols,nrows,crs,toptobottom,tiled,dim_order
13997,gridmet,None,http://thredds.northwestknowledge.net:8080/thr...,opendap,daily_maximum_temperature,tmmx,tmmx,K,None,None,...,49.4,25.066667,0.041667,0.041667,1386.0,585.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,False,,TYX


In [21]:
# Create a dictionary of parameter dataframes for each variable
tvars = ["tmmx"]
cat_params = [cat.query("id == @_id & variable == @_var").to_dict(orient="records")[0] for _var in tvars]

cat_dict = dict(zip(tvars, cat_params))

# Output an example of the cat_param.json entry for "aet".
cat_dict.get("tmmx")

{'id': 'gridmet',
 'asset': None,
 'URL': 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_met_tmmx_1979_CurrentYear_CONUS.nc',
 'type': 'opendap',
 'varname': 'daily_maximum_temperature',
 'variable': 'tmmx',
 'description': 'tmmx',
 'units': 'K',
 'model': None,
 'ensemble': None,
 'scenario': None,
 'T_name': 'day',
 'duration': '1979-01-01/..',
 'interval': '1 days',
 'nT': nan,
 'X_name': 'lon',
 'Y_name': 'lat',
 'X1': -124.76666663333334,
 'Xn': -67.05833330000002,
 'Y1': 49.400000000000006,
 'Yn': 25.066666666666666,
 'resX': 0.041666666666666664,
 'resY': 0.04166666666666668,
 'ncols': 1386.0,
 'nrows': 585.0,
 'crs': '+proj=longlat +a=6378137 +f=0.00335281066474748 +pm=0 +no_defs',
 'toptobottom': False,
 'tiled': '',
 'dim_order': 'TYX'}

In [22]:
user_data = ClimRCatData(
    cat_dict=cat_dict,
    f_feature=gdf,
    id_feature='nhm_id',
    period=["1980-01-01", "1980-12-31"]
)

wght_gen = WeightGen(
    user_data=user_data,
    method="serial",
    output_file=root_dir + "GIS/uc_weights.csv",
    weight_gen_crs=6931
)

wghts = wght_gen.calculate_weights(intersections=False)

Using serial engine
grid cells generated in 0.0180 seconds
Data preparation finished in 0.8055 seconds
Reprojecting to epsg:EPSG:6931 finished in 0.02 second(s)
Validating polygons
     - validating source polygons
     - fixing 0 invalid polygons.
     - validating target polygons
     - fixing 1 invalid polygons.
Validate polygons finished in 0.0037 seconds
Intersections finished in 0.0099 seconds
Weight gen finished in 0.0147 seconds


In [28]:
root_path = Path(root_dir)
daily_path = root_path / "daily"
daily_path.mkdir(exist_ok=True)
daily_path = root_path / "daily/input"
daily_path.mkdir(exist_ok=True)
daily_path = root_path / "daily/restart"
daily_path.mkdir(exist_ok=True)
daily_path = root_path / "daily/output"
daily_path.mkdir(exist_ok=True)
frcst_path = root_path / "forecast/"
frcst_path.mkdir(exist_ok=True)
frcst_path = root_path / "forecast/input"
frcst_path.mkdir(exist_ok=True)
frcst_path = root_path / "forecast/restart"
frcst_path.mkdir(exist_ok=True)
frcst_path = root_path / "forecast/output"
frcst_path.mkdir(exist_ok=True)

In [31]:
import os
import subprocess
prms_path = '/home/rmcd/prms_5.2.1_linux/bin/prms'
project_root = '/mnt/d/oNHM/NHM_PRMS_TEST'
# hard-coded by looking at bandit.control file 
start_time = "1979-06-01"
end_time = "2021-12-31"
os.chdir(project_root)
command = [
   prms_path,
   "-set", "save_vars_to_file", "1",
   "-set", "var_save_file", f"{project_root}/daily/restart/{end_time}.restart",
   "-C", f"{project_root}/control.default.bandit"
]
subprocess.run(command, check=True)


Control variable save_vars_to_file set to 1.


Control variable var_save_file set to /mnt/d/oNHM/NHM_PRMS_TEST/daily/restart/2021-12-31.restart.






MMS - Warning: Variable runoff not declared at line number 9 in
sf_data
runoff 1

         set in the Parameter File. Module default values are being used.





                         U.S. Geological Survey
               Precipitation-Runoff Modeling System (PRMS)
                        Version 5.2.1 02/08/2022

        Process            Available Modules
--------------------------------------------------------------------
  Basin Definition: basin
    Cascading Flow: cascade
  Time Series Data: obs, water_use_read, dynamic_param_read
   Potet Solar Rad: soltab
  Temperature Dist: temp_1sta, temp_laps, temp_dist2, climate_hru,
                    temp_map
       Precip Dist: precip_1sta, precip_laps, precip_dist2,
                    climate_hru, precip_map
Temp & Precip Dist: xyz_dist, ide_dist
    Solar Rad Dist: ccsolrad, ddsolrad, climate_hru
Transpiration Dist: transp_tindex, climate_hru, transp_frost
      Potential ET: potet_hamon, potet_jh, potet_pan, climate_hru,
                    potet_hs, potet_pt, potet_pm, potet_pm_sta
      Interception: intcp
Snow & Glacr Dynam: snowcomp, glacr_melt
    Surface Runoff: srunoff_smidx, s


WARNING, date of end_time reached the last date in the Data File 
         simulation stopped on: 2021 12 31 


CompletedProcess(args=['/home/rmcd/prms_5.2.1_linux/bin/prms', '-set', 'save_vars_to_file', '1', '-set', 'var_save_file', '/mnt/d/oNHM/NHM_PRMS_TEST/daily/restart/2021-12-31.restart', '-C', '/mnt/d/oNHM/NHM_PRMS_TEST/control.default.bandit'], returncode=0)

## Rename control file and update the control file for a somplete set of output
- Rename the control file to `NHM_TEST.control`
- replace the equivalent lines in the new control file with the following
```
####
nhruOutVars
1
1
66
####
nhruOutVar_names
66
4
soil_moist_tot
unused_potet
hru_ppt
hru_rain
hru_snow
potet
hru_actet
swrad
tmaxf
tminf
tavgf
dprst_evap_hru
dprst_insroff_hru
dprst_seep_hru
dprst_vol_open_frac
dprst_vol_open
dprst_sroff_hru
dprst_area_open
dprst_stor_hru
ssres_flow
slow_flow
dunnian_flow
hortonian_flow
gwres_flow
hru_sroffi
hru_sroffp
sroff
hru_streamflow_out
hru_lateral_flow
pref_flow
hru_outflow
hru_intcpevap
hru_intcpstor
hru_impervevap
hru_impervstor
net_ppt
net_rain
net_snow
contrib_fraction
albedo
pk_depth
pk_temp
pkwater_equiv
snowcov_area
pk_ice
pk_precip
snow_evap
snow_free
snowmelt
freeh2o
soil_to_ssr
soil_to_gw
soil_rechr
cap_waterin
infil
perv_actet
pref_flow_stor
recharge
slow_stor
soil_moist
gwres_stor
gwres_in
prmx
transp_on
newsnow
intcp_on
```
and

```
####
nhruOut_format
1
1
4
```

and
```
####
nsegmentOutVars
1
1
10
####
nsegmentOutVar_names
10
4
seg_outflow
seg_width
seg_tave_water
seg_tave_upstream
seg_tave_air
seg_tave_gw
seg_tave_sroff
seg_tave_lat
seg_shade
seg_potet
```
